In [ ]:
pip install chromadb

In [ ]:
pip install langchain

In [ ]:
pip install openai==1.3.7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
#import getpass

os.environ["OPENAI_API_BASE"] = 'https://oneapi.xty.app/v1'
os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/research/AI agent/100")

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
from langchain.schema.document import Document


documents =  Document(page_content='people, bus, road', metadata={"source": "local"})

In [ ]:
db = Chroma.from_documents([documents], OpenAIEmbeddings(), collection_metadata={"hnsw:space": "ip"})

In [ ]:
query = "people"
docs = db.similarity_search_with_relevance_scores(query,k=1)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('51d3bc1035b34be8c39a9f06b41a07f2f3f8_1.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
#db = Chroma.from_documents(documents, OpenAIEmbeddings())
db = Chroma.from_documents(documents, OpenAIEmbeddings(), collection_metadata={"hnsw:space": "ip"})

In [ ]:
query = "My input smiles is CN1C2=CC=CC(C3=CC=C4N=CC=CC4=C3)=C2C=N1, please help me generate python code to get 2-D similar molecule using PubChem API."
docs = db.similarity_search_with_relevance_scores(query,k=50)

In [ ]:
query = "How to perform 2-D similarity search"
docs = db.similarity_search_with_score(query,k=50)

### inner product

In [ ]:
query = "How to perform 2-D similarity search"
docs = db.similarity_search_with_relevance_scores(query,k=50)

In [ ]:
for i in range(len(docs)):
  print(docs[i][1])

0.8118360042572021
0.8050254583358765
0.7794522047042847
0.7561227679252625
0.7367781400680542
0.7359175682067871
0.7245218753814697
0.7208773493766785
0.7184016108512878
0.7147347927093506
0.7101709246635437
0.7099320888519287
0.7095848321914673
0.7010208964347839
0.6938539743423462
0.6898643970489502
0.6881193518638611
0.6845148205757141


### cosine distance

In [ ]:
query = "How to perform 2-D similarity search"
docs = db.similarity_search_with_relevance_scores(query,k=50)

In [ ]:
for i in range(len(docs)):
  print(docs[i][1])

0.8118367791175842
0.8050261735916138
0.7794528603553772
0.7561233043670654
0.7367786765098572
0.7359180450439453
0.7245229482650757
0.7208778262138367
0.7184019088745117
0.7147352695465088
0.7101714611053467
0.7099325656890869
0.7095852494239807
0.7010214924812317
0.693854808807373
0.6898650527000427
0.6881195306777954
0.6845157742500305


### L2 distance

In [ ]:
query = "How to perform 2-D similarity search"
docs = db.similarity_search_with_relevance_scores(query,k=50)

In [ ]:
for i in range(len(docs)):
  print(docs[i][1])

0.7338959674170704
0.7242643167142699
0.6880986539161025
0.6551053001116529
0.6277479914774741
0.6265308747861764
0.610415100027369
0.6052609619229716
0.601759359601838
0.5965739485358458
0.5901194537143419
0.5897817731640727
0.5892908466726178
0.5771794018439818
0.5670438855672334
0.5614021084255651
0.5589342840047067
0.5538368334115567


In [ ]:
print(docs[0][0].page_content)

search, the similarity between conformers are evaluated                    Note that the ‘Threshold’ option takes the integer value of
with the shape-Tanimoto (ST) and color-Tanimoto (CT)                       80 (multiplying 0.80 by 100).
scores, which quantify the similarity between their conform-                  For most queries, the new fast chemical search approach
ers in 3-D shape and functional group orientations, respec-                in PUG-REST provides a dramatic benefit of programmatic
tively (8,9). The ST and CT scores are calculated using the                simplicity, speed, and convenience. In contrast, it is worth
Gaussian-shape overlay method by Grant and Pickup (10–                     noting that some complicated chemical structure searches
12), and implemented in the Rapid Overlay of Chemical                      will not complete rapidly, and may fail as a result. Although
Structures (ROCS) (13). Up to ten conformers per com-                      one may try the

# prompt to GPT

In [ ]:
file_list = []

In [ ]:
import re

def extract_and_save_python_code(text, name, repeated_times):

    # Regular expression pattern for extracting Python code blocks
    pattern = r"```python(.*?)```"
    # Use re.DOTALL to match across multiple lines
    matches = re.findall(pattern, text, re.DOTALL)

    # Save each extracted code block to a separate .py file
    for i, code_block in enumerate(matches, 1):
        file_name = f"extracted_code_block_{name}_{repeated_times}_{i}.py"
        with open(file_name, 'w') as file:
            file.write(code_block.strip())
        print(f"Saved Python code to {file_name}")

    file_list.append(file_name)

    return matches

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://oneapi.xty.app/v1",
    api_key="sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
)

## zeroshot

In [ ]:
def prompt_function_zeroshot():
  completion = client.chat.completions.create(
  model="gpt-4-32k",
  messages=[{"role": "system", "content": "You are a helpful assistant on chemistry."},
    {"role": "user", "content": '''My input smiles is "CN1C2=CC=CC(C3=CC=C4N=CC=CC4=C3)=C2C=N1", please help me generate python code to get similar molecule using PubChem API.
    '''}
  ],
  stream = True
)
  return completion

In [ ]:
for i in range(10):
  completion = prompt_function_zeroshot()
  text_print = ""

  for chunk in completion:
    if hasattr(chunk.choices[0].delta, 'content') and chunk.choices[0].delta.content is not None:
      text_print = text_print + chunk.choices[0].delta.content

  extracted_code_blocks = extract_and_save_python_code(text_print, 'zeroshot', i)

Saved Python code to extracted_code_block_zeroshot_0_1.py
Saved Python code to extracted_code_block_zeroshot_1_1.py
Saved Python code to extracted_code_block_zeroshot_2_1.py
Saved Python code to extracted_code_block_zeroshot_3_1.py
Saved Python code to extracted_code_block_zeroshot_4_1.py
Saved Python code to extracted_code_block_zeroshot_5_1.py
Saved Python code to extracted_code_block_zeroshot_6_1.py
Saved Python code to extracted_code_block_zeroshot_6_2.py
Saved Python code to extracted_code_block_zeroshot_7_1.py
Saved Python code to extracted_code_block_zeroshot_8_1.py
Saved Python code to extracted_code_block_zeroshot_9_1.py


## all document

In [ ]:
def prompt_function_fewshot(content):
  completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[{"role": "system", "content": "You are a helpful assistant on chemistry."},
    {"role": "user", "content": f'''My input smiles is "CN1C2=CC=CC(C3=CC=C4N=CC=CC4=C3)=C2C=N1", please help me generate python code to get similar molecule using PubChem API. You can refer to: '{content}'.
    '''}
  ],
  stream = True
)
  return completion

In [ ]:
from tqdm import tqdm

In [ ]:
import time

time_list = []
for i in tqdm(range(10)):
  start_time = time.time()

  completion = prompt_function_fewshot(str(raw_documents[0]))
  text_print = ""

  for chunk in completion:
    if hasattr(chunk.choices[0].delta, 'content') and chunk.choices[0].delta.content is not None:
      text_print = text_print + chunk.choices[0].delta.content

  end_time = time.time()
  execution_time = end_time - start_time
  time_list.append(execution_time)

  extracted_code_blocks = extract_and_save_python_code(text_print, 'alldocument', i)
  print(time_list)

 10%|█         | 1/10 [00:16<02:28, 16.54s/it]

Saved Python code to extracted_code_block_alldocument_0_1.py
[16.529057025909424]


 20%|██        | 2/10 [01:06<04:51, 36.49s/it]

Saved Python code to extracted_code_block_alldocument_1_1.py
[16.529057025909424, 50.45263171195984]


 30%|███       | 3/10 [01:25<03:18, 28.32s/it]

...

Saved Python code to extracted_code_block_alldocument_7_1.py
[16.529057025909424, 50.45263171195984, 18.59399175643921, 32.66144776344299, 20.767174005508423, 41.516963958740234, 37.77752065658569, 39.645495653152466]


 90%|█████████ | 9/10 [05:02<00:38, 38.52s/it]

Saved Python code to extracted_code_block_alldocument_8_1.py
[16.529057025909424, 50.45263171195984, 18.59399175643921, 32.66144776344299, 20.767174005508423, 41.516963958740234, 37.77752065658569, 39.645495653152466, 44.89113712310791]


100%|██████████| 10/10 [05:26<00:00, 32.66s/it]

Saved Python code to extracted_code_block_alldocument_9_1.py
[16.529057025909424, 50.45263171195984, 18.59399175643921, 32.66144776344299, 20.767174005508423, 41.516963958740234, 37.77752065658569, 39.645495653152466, 44.89113712310791, 23.643864393234253]


## topmatch

In [ ]:
#time_list = []
for i in tqdm(range(10)):
  #start_time = time.time()

  completion = prompt_function_fewshot(docs[0][0].page_content)
  text_print = ""

  for chunk in completion:
    if hasattr(chunk.choices[0].delta, 'content') and chunk.choices[0].delta.content is not None:
      text_print = text_print + chunk.choices[0].delta.content

  #end_time = time.time()
  #execution_time = end_time - start_time
  #time_list.append(execution_time)

  extracted_code_blocks = extract_and_save_python_code(text_print, 'topmatch', i)
  #print(time_list)

 10%|█         | 1/10 [00:17<02:34, 17.12s/it]

Saved Python code to extracted_code_block_topmatch_0_1.py


 20%|██        | 2/10 [00:53<03:48, 28.56s/it]

Saved Python code to extracted_code_block_topmatch_1_1.py


 30%|███       | 3/10 [01:24<03:28, 29.72s/it]

...

Saved Python code to extracted_code_block_topmatch_7_1.py


 90%|█████████ | 9/10 [04:19<00:28, 28.76s/it]

Saved Python code to extracted_code_block_topmatch_8_1.py


100%|██████████| 10/10 [05:16<00:00, 31.64s/it]

Saved Python code to extracted_code_block_topmatch_9_1.py


# random

In [ ]:
import random

random_integer = random.randint(0, 17)

print(random_integer)

6


In [ ]:
time_list = []
for i in tqdm(range(1)):
  start_time = time.time()

  random_integer = random.randint(0, 17)
  random_slice = documents[random_integer].page_content
  completion = prompt_function_fewshot(random_slice)
  text_print = ""

  end_time = time.time()

  for chunk in completion:
    if hasattr(chunk.choices[0].delta, 'content') and chunk.choices[0].delta.content is not None:
      text_print = text_print + chunk.choices[0].delta.content

  execution_time = end_time - start_time
  time_list.append(execution_time)

  extracted_code_blocks = extract_and_save_python_code(text_print, 'random', 9)
  print(time_list)

100%|██████████| 1/1 [00:52<00:00, 52.32s/it]

Saved Python code to extracted_code_block_random_9_1.py
[1.535128116607666]
